In [2]:
import pandas as pd
import numpy as np
import json
import ast
import nltk

# file to analyze = steam_games 

### read and info

In [3]:
# list 'output_steam_games.json'
rows = []
with open ('data/steam_games.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        rows.append(json.loads(line))

# save to df
df_steam_games = pd.DataFrame(rows)

In [481]:
df_steam_games.dropna(how='all', inplace=True)
df_steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88314,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [482]:
df_steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 3.4+ MB


In [483]:
df_steam_games.isna().sum()

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

In [484]:
# fill na values of 'title' with 'app_name' values
df_steam_games['title'].fillna(df_steam_games['app_name'], inplace=True)

### genres, tags

In [485]:
df_steam_games.isna().sum()

publisher       8052
genres          3283
app_name           2
title              2
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

In [486]:
df_steam_games['genres'].fillna(df_steam_games['tags'], inplace=True)


In [487]:
# Fill NaN values with an empty list
df_steam_games['genres'] = df_steam_games['genres'].fillna('')
df_steam_games['tags'] = df_steam_games['tags'].fillna('')

# Define a function to handle the conversion
def convert_to_string(x):
    if isinstance(x, list):
        return x
    else:
        return x.split(', ')
    
# Apply this function to the 'genres' and 'tags' columns
df_steam_games['genres'] = df_steam_games['genres'].apply(convert_to_string)
df_steam_games['tags'] = df_steam_games['tags'].apply(convert_to_string)

# Concatenate the 'genres' and 'tags' columns
df_steam_games['genres'] = df_steam_games['genres'] + df_steam_games['tags']

# Convert to a set to remove duplicates, then convert back to a list
df_steam_games['genres'] = df_steam_games['genres'].apply(lambda x: list(set(x)))

In [488]:
# Initialize an empty set to store the unique genres
unique_genres = set()

# Iterate over the 'genres' column
for genres_list in df_steam_games['genres']:
    # Add the genres from this row to the set
    unique_genres.update(genres_list)

# Now 'unique_genres' contains all unique genres
unique_genres

{'',
 '1980s',
 "1990's",
 '2.5D',
 '2D',
 '2D Fighter',
 '360 Video',
 '3D Platformer',
 '3D Vision',
 '4 Player Local',
 '4X',
 '6DOF',
 'Abstract',
 'Accounting',
 'Action',
 'Action RPG',
 'Action-Adventure',
 'Adventure',
 'Agriculture',
 'Aliens',
 'Alternate History',
 'America',
 'Animation & Modeling',
 'Animation &amp; Modeling',
 'Anime',
 'Arcade',
 'Arena Shooter',
 'Artificial Intelligence',
 'Assassin',
 'Asynchronous Multiplayer',
 'Atmospheric',
 'Audio Production',
 'Base Building',
 'Based On A Novel',
 'Basketball',
 'Batman',
 "Beat 'em up",
 'Benchmark',
 'Bikes',
 'Blood',
 'Board Game',
 'Bowling',
 'Building',
 'Bullet Hell',
 'Bullet Time',
 'CRPG',
 'Capitalism',
 'Card Game',
 'Cartoon',
 'Cartoony',
 'Casual',
 'Character Action Game',
 'Character Customization',
 'Chess',
 'Choices Matter',
 'Choose Your Own Adventure',
 'Cinematic',
 'City Builder',
 'Class-Based',
 'Classic',
 'Clicker',
 'Co-op',
 'Co-op Campaign',
 'Cold War',
 'Colorful',
 'Comedy',
 

### ID, app_name

In [489]:
# find missing id and app_name
missing = df_steam_games[df_steam_games[['id', 'app_name']].isna().any(axis=1)]
missing

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88384,NaN,[],NaN,NaN,http://store.steampowered.com/,NaN,[],NaN,NaN,19.99,False,NaN,NaN
90890,NaN,"[Indie, Action]",NaN,NaN,http://store.steampowered.com/app/317160/_/,2014-08-26,"[Action, Indie]",http://steamcommunity.com/app/317160/reviews/?...,"[Single-player, Game demo]",NaN,False,317160,NaN
119271,"Warner Bros. Interactive Entertainment, Feral ...","[Singleplayer, Third Person, Controller, Fight...",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [490]:
# Extract the id of the URL for Batman
df_steam_games.loc[
    df_steam_games['title'] == 'Batman: Arkham City - Game of the Year Edition', 'id'
    ] = df_steam_games['url'].apply(lambda x: x[-6:])


In [491]:
# Delete 
df_steam_games = df_steam_games[df_steam_games['url'] != 'http://store.steampowered.com/']
df_steam_games = df_steam_games[df_steam_games['id'] != '317160']

In [492]:
# checking missing again
missing = df_steam_games[df_steam_games[['id', 'app_name']].isna().any(axis=1)]
missing

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer


In [493]:
# Check content for some columns, select genres to work with
print(df_steam_games['genres'])

88310         [Simulation, Indie, Casual, Action, Strategy]
88311     [Design & Illustration, Card Game, Dark Fantas...
88312     [Simulation, Indie, Sports, Casual, Multiplaye...
88313                           [Adventure, Action, Casual]
88314                       [Sports, Indie, Action, Casual]
                                ...                        
120440                [Simulation, Indie, Casual, Strategy]
120441                            [Indie, Casual, Strategy]
120442                          [Simulation, Indie, Racing]
120443    [Indie, Atmospheric, Casual, Singleplayer, Puz...
120444    [Simulation, Indie, Early Access, VR, Adventur...
Name: genres, Length: 32133, dtype: object


### price

In [494]:
df_steam_games['price'].unique()

array([4.99, 'Free To Play', 'Free to Play', 0.99, 2.99, 3.99, 9.99,
       18.99, 29.99, nan, 'Free', 10.99, 1.59, 14.99, 1.99, 59.99, 8.99,
       6.99, 7.99, 39.99, 19.99, 7.49, 12.99, 5.99, 2.49, 15.99, 1.25,
       24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99,
       16.99, 79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99,
       149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 64.99,
       'Free Mod', 54.99, 74.99, 'Install Theme', 0.89, 'Third-party',
       0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0, 5.49, 23.99, 49.0, 20.99,
       10.93, 1.39, 'Free HITMAN™ Holiday Pack', 36.99, 4.49, 2.0, 4.0,
       9.0, 234.99, 1.95, 1.5, 199.0, 189.0, 6.66, 27.99, 10.49, 129.99,
       179.0, 26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33, 199.99,
       22.99, 320.0, 38.85, 71.7, 59.95, 995.0, 27.49, 3.39, 6.0, 19.95,
       499.99, 16.06, 4.68, 131

In [495]:
df_steam_games.dropna(subset='price', inplace=True)

In [496]:
df_steam_games['price'].isnull().sum()

0

In [497]:
df_steam_games['price'] = df_steam_games['price'].replace(r'.*Free.*', '0.00', regex=True)
df_steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Simulation, Indie, Casual, Action, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
88311,"Making Fun, Inc.","[Design & Illustration, Card Game, Dark Fantas...",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,643980,Secret Level SRL
88312,Poolians.com,"[Simulation, Indie, Sports, Casual, Multiplaye...",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,False,670290,Poolians.com
88313,彼岸领域,"[Adventure, Action, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域
88314,NaN,"[Sports, Indie, Action, Casual]",Log Challenge,Log Challenge,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Simulation, Indie, Casual, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Indie, Casual, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Simulation, Indie, Racing]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Indie, Atmospheric, Casual, Singleplayer, Puz...",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


In [498]:
df_steam_games['price'].unique()

array([4.99, '0.00', 0.99, 2.99, 3.99, 9.99, 18.99, 29.99, 10.99, 1.59,
       14.99, 1.99, 59.99, 8.99, 6.99, 7.99, 39.99, 19.99, 7.49, 12.99,
       5.99, 2.49, 15.99, 1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99,
       34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99, 16.99, 79.99,
       49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99, 149.99, 771.71,
       'Install Now', 21.99, 89.99, 'Play WARMACHINE: Tactics Demo', 0.98,
       139.92, 4.29, 64.99, 54.99, 74.99, 'Install Theme', 0.89,
       'Third-party', 0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0, 5.49,
       23.99, 49.0, 20.99, 10.93, 1.39, 36.99, 4.49, 2.0, 4.0, 9.0,
       234.99, 1.95, 1.5, 199.0, 189.0, 6.66, 27.99, 10.49, 129.99, 179.0,
       26.99, 399.99, 31.99, 399.0, 20.0, 40.0, 3.33, 199.99, 22.99,
       320.0, 38.85, 71.7, 59.95, 995.0, 27.49, 3.39, 6.0, 19.95, 499.99,
       16.06, 4.68, 131.4, 44.98, 202.76, 1.0, 2.3, 0.95, 172.24, 249.99,
       2.97, 10.96, 10.0, 30.0, 2.66, 6.48, 19.29, 11.15, 18.9, 2.89,
  

In [499]:
# Convert all non-float values to '0.00'
df_steam_games['price'] = pd.to_numeric(df_steam_games['price'], errors='coerce')
df_steam_games['price'].fillna('0.00', inplace=True)

# Replace 'Starting at $...' values with the price after it
# df_steam_games['price'] = df_steam_games['price'].replace(r'^Starting at \$([0-9\.]+).*$', r'\1', regex=True)
# df_steam_games.loc[df_steam_games['price'].str.startswith('Starting at $'), 'price'] = df_steam_games['price'].str.extract('Starting at \$([0-9\.]+)', expand=False)
# df_steam_games['price'] = df_steam_games['price'].replace(r'^Starting at $([0-9\.]+).*$', lambda x: x.group(1), regex=True)

# Honorless solution.......
df_steam_games['price'] = df_steam_games['price'].replace('Starting at $499.00', 499.00)
df_steam_games['price'] = df_steam_games['price'].replace('Starting at $449.00', 449.00)

In [500]:
# Convert the 'price' column to a small float type
df_steam_games['price'] = pd.to_numeric(df_steam_games['price'], errors='coerce').astype('float16')

In [501]:
# Replace '0.00' string with 0.0 float
df_steam_games['price'] = df_steam_games['price'].replace('0.00', 0.0)

# Convert the 'price' column to a small float type
df_steam_games['price'] = pd.to_numeric(df_steam_games['price'], errors='coerce').astype('float16')

In [502]:
df_steam_games['price'].unique()

array([4.99, 0.00, 0.99, 2.99, 3.99, 9.99, 18.98, 29.98, 10.99, 1.59,
       14.99, 1.99, 60.00, 8.99, 6.99, 7.99, 40.00, 19.98, 7.49, 12.99,
       5.99, 2.49, 15.99, 1.25, 24.98, 17.98, 62.00, 3.49, 11.99, 13.99,
       35.00, 74.75, 1.49, 33.00, 100.00, 14.95, 70.00, 16.98, 80.00,
       50.00, 5.00, 45.00, 13.98, 29.95, 120.00, 110.00, 150.00, 771.50,
       21.98, 90.00, 0.98, 139.88, 4.29, 65.00, 55.00, 75.00, 0.89, 0.50,
       300.00, 1.29, 3.00, 15.00, 5.49, 23.98, 49.00, 20.98, 10.93, 1.39,
       37.00, 4.49, 2.00, 4.00, 9.00, 235.00, 1.95, 1.50, 199.00, 189.00,
       6.66, 27.98, 10.49, 130.00, 179.00, 26.98, 400.00, 31.98, 399.00,
       20.00, 3.33, 200.00, 22.98, 320.00, 38.84, 71.69, 59.94, 995.00,
       27.48, 3.39, 6.00, 19.95, 500.00, 16.06, 4.68, 131.38, 44.97,
       202.75, 1.00, 2.30, 0.95, 172.25, 250.00, 2.97, 10.96, 10.00,
       30.00, 2.66, 6.48, 19.30, 11.15, 18.91, 2.89, 99.00, 87.94, 599.00,
       8.98, 9.69, 0.49, 9.98, 9.95, 7.00, 12.89, 6.49, 1.87, 

In [503]:
np.set_printoptions(formatter={'float_kind':'{:0.2f}'.format})
np.round(df_steam_games['price'].unique(), 2)

/home/p/Code/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: RuntimeWarning: overflow encountered in multiply
  return bound(*args, **kwds)


array([4.99, 0.00, 0.99, 2.99, 3.99, 9.99, 18.98, 29.98, 10.99, 1.59,
       14.99, 1.99, 60.00, 8.99, 6.99, 7.99, 40.00, 19.98, 7.49, 12.99,
       5.99, 2.49, 15.99, 1.25, 24.98, 17.98, 62.00, 3.49, 11.99, 13.99,
       35.00, 74.75, 1.49, 33.00, 100.00, 14.95, 70.00, 16.98, 80.00,
       50.00, 5.00, 45.00, 13.98, 29.95, 120.00, 110.00, 150.00, inf,
       21.98, 90.00, 0.98, 139.88, 4.29, 65.00, 55.00, 75.00, 0.89, 0.50,
       300.00, 1.29, 3.00, 15.00, 5.49, 23.98, 49.00, 20.98, 10.93, 1.39,
       37.00, 4.49, 2.00, 4.00, 9.00, 235.00, 1.95, 1.50, 199.00, 189.00,
       6.66, 27.98, 10.49, 130.00, 179.00, 26.98, 400.00, 31.98, 399.00,
       20.00, 3.33, 200.00, 22.98, 320.00, 38.84, 71.69, 59.91, inf,
       27.48, 3.39, 6.00, 19.95, 499.75, 16.06, 4.68, 131.38, 44.97,
       202.75, 1.00, 2.30, 0.95, 172.38, 249.88, 2.97, 10.96, 10.00,
       30.00, 2.66, 6.48, 19.30, 11.15, 18.91, 2.89, 99.06, 87.94, 599.00,
       8.98, 9.69, 0.49, 9.98, 9.96, 7.00, 12.89, 6.49, 1.87, 43.00,

In [504]:
df_steam_games['price'].unique()

array([4.99, 0.00, 0.99, 2.99, 3.99, 9.99, 18.98, 29.98, 10.99, 1.59,
       14.99, 1.99, 60.00, 8.99, 6.99, 7.99, 40.00, 19.98, 7.49, 12.99,
       5.99, 2.49, 15.99, 1.25, 24.98, 17.98, 62.00, 3.49, 11.99, 13.99,
       35.00, 74.75, 1.49, 33.00, 100.00, 14.95, 70.00, 16.98, 80.00,
       50.00, 5.00, 45.00, 13.98, 29.95, 120.00, 110.00, 150.00, 771.50,
       21.98, 90.00, 0.98, 139.88, 4.29, 65.00, 55.00, 75.00, 0.89, 0.50,
       300.00, 1.29, 3.00, 15.00, 5.49, 23.98, 49.00, 20.98, 10.93, 1.39,
       37.00, 4.49, 2.00, 4.00, 9.00, 235.00, 1.95, 1.50, 199.00, 189.00,
       6.66, 27.98, 10.49, 130.00, 179.00, 26.98, 400.00, 31.98, 399.00,
       20.00, 3.33, 200.00, 22.98, 320.00, 38.84, 71.69, 59.94, 995.00,
       27.48, 3.39, 6.00, 19.95, 500.00, 16.06, 4.68, 131.38, 44.97,
       202.75, 1.00, 2.30, 0.95, 172.25, 250.00, 2.97, 10.96, 10.00,
       30.00, 2.66, 6.48, 19.30, 11.15, 18.91, 2.89, 99.00, 87.94, 599.00,
       8.98, 9.69, 0.49, 9.98, 9.95, 7.00, 12.89, 6.49, 1.87, 

### developer, publisher


In [505]:
df_steam_games['developer'].fillna(df_steam_games['publisher'], inplace=True)

In [506]:
df_steam_games.dropna(subset=['developer'], inplace=True)

In [507]:
df_steam_games.isna().sum()

publisher       4680
genres             0
app_name           0
title              0
url                0
release_date       4
tags               0
reviews_url        1
specs            280
price              0
early_access       0
id                 0
developer          0
dtype: int64

### Release_date

In [508]:
df_steam_games['release_date'].unique()

array(['2018-01-04', '2017-07-24', '2017-12-07', ..., '2003-11-01',
       '2004-03-16', '2004-03-01'], dtype=object)

In [509]:
# Encuentra las filas con NaN en 'release_date'
# df_steam_games[df_steam_games['release_date'].isna()]

In [510]:
# extract the year of the release_date
df_steam_games['release_date'] = df_steam_games['release_date'].str.extract(r'(\d{4})')

In [511]:
df_steam_games['release_date'].unique()

array(['2018', '2017', '1997', '1998', '2016', '2006', '2005', '2003',
       '2007', '2002', '2000', '1995', '1996', '1994', '2001', '1993',
       '2004', '2008', '2009', '1999', '1992', '1989', '2010', '2011',
       '2013', '2012', '2014', nan, '1983', '1984', '2015', '1990',
       '1988', '1991', '1987', '1986', '2021', '2019', '1985'],
      dtype=object)

In [512]:
df_steam_games.dropna(subset=['release_date'], inplace=True)

### title, app_name

In [513]:
df_steam_games.isna().sum()

publisher       4679
genres             0
app_name           0
title              0
url                0
release_date       0
tags               0
reviews_url        1
specs            280
price              0
early_access       0
id                 0
developer          0
dtype: int64

In [514]:
df_steam_games['title'].unique()
# df_steam_games['app_name'].unique()

array(['Lost Summoner Kitty', 'Ironbound', 'Real Pool 3D - Poolians', ...,
       'LOGistICAL: South Africa', 'Russian Roads', 'EXIT 2 - Directions'],
      dtype=object)

In [515]:
# drop the columns that dont have importance to the model 
df_steam_games.drop(['publisher', 'title', 'url', 'tags', 'reviews_url', 'specs', 'early_access'], axis=1, inplace=True)

In [516]:
df_steam_games

,genres,app_name,release_date,price,id,developer
88310,"[Simulation, Indie, Casual, Action, Strategy]",Lost Summoner Kitty,2018,4.99,761140,Kotoshiro
88311,"[Design & Illustration, Card Game, Dark Fantas...",Ironbound,2018,0.00,643980,Secret Level SRL
88312,"[Simulation, Indie, Sports, Casual, Multiplaye...",Real Pool 3D - Poolians,2017,0.00,670290,Poolians.com
88313,"[Adventure, Action, Casual]",弹炸人2222,2017,0.99,767400,彼岸领域
88315,"[Simulation, FPS, Sniper, Third Person, Shoote...",Battle Royale Trainer,2018,3.99,772540,Trickjump Games Ltd
...,...,...,...,...,...,...
120439,"[Indie, Casual, Violent, Adventure, Action]",Kebab it Up!,2018,1.99,745400,Bidoniera Games
120440,"[Simulation, Indie, Casual, Strategy]",Colony On Mars,2018,1.99,773640,"Nikita ""Ghost_RUS"""
120441,"[Indie, Casual, Strategy]",LOGistICAL: South Africa,2018,4.99,733530,Sacada
120442,"[Simulation, Indie, Racing]",Russian Roads,2018,1.99,610660,Laush Dmitriy Sergeevich


In [517]:
df_steam_games['app_name'] = df_steam_games['app_name'].str.lower()

In [518]:
df_steam_games.to_csv('df_steam_games.csv', index=False)

In [ ]:
df_steam_games.write.parquet("steam_games.parquet", compression="gzip")